In [1]:
sys.path.append('C:/Users/nyasha/Desktop/Masters-Nyasha/Models')

import sys

import torch
import numpy as np
import torch.optim as optim
import torchvision.transforms as transforms

from CNN import ConvNet1
from training_functions import *

In [2]:
batch_size = 128
num_epochs = 25

In [3]:
# Extracting the training, validation and testing data
compressed_file_path = "C:/Users/nyasha/Desktop/Masters-Nyasha/Processed Data/processed_data.npz"
x_train, x_test, x_val, y_train, y_test, y_val = get_train_test_data(compressed_file_path)

In [4]:
# Check if files have been imported correctly
print(x_train.shape)  
print(x_test.shape)
print(x_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(11035, 128, 44)
(3679, 128, 44)
(3679, 128, 44)
(11035,)
(3679,)
(3679,)


In [5]:
# Creating the data loaders
train_loader, val_loader, test_loader = createDataloaders(x_train, y_train, x_val, y_val, x_test, y_test, batch_size)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32 * 30 * 9, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 6)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 30 * 9)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [38]:
import torch.nn as nn
import torch.nn.functional as F

class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2, 1)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32 * 16 * 5, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 6)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 32 * 16 * 5)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [24]:
class ConvNet1(nn.Module):
    def __init__(self):
        super(ConvNet1, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(10, 20, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(20,20, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Sequential(
            nn.Linear(20 * 1 *8, 128),
            nn.ReLU())
        self.fc2 = nn.Linear(128, 4)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [7]:
# Select a model from ../Models
model_1  = Net()

In [41]:
data = iter(train_loader)
images, _ = next(data)

In [8]:
device = get_device() 
model_on_device = model_1.to(device) # Move model to the current device

Running on the GPU


In [43]:
images = images.to(device, dtype=torch.float)
x = model_1.pool(F.relu(model_1.conv1(images)))
x = model_1.pool(F.relu(model_1.conv2(x)))
x = model_1.pool(F.relu(model_1.conv3(x)))
x.shape

torch.Size([128, 32, 16, 5])

In [9]:
class_weights = get_class_weights(y_train, device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.SGD(model_on_device.parameters(), lr=0.01, momentum=0.9)

C:\Users\nyasha\anaconda3\envs\masters\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=['2_walking' 'clutter/noise' 'running' 'sphere_swing' 'vehicle' 'walking'], y=['walking' 'vehicle' 'sphere_swing' ... 'sphere_swing' 'walking' 'vehicle'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [10]:
class_weights

tensor([0.9888, 0.9995, 1.3118, 1.4892, 0.6809, 0.9210], device='cuda:0')

In [11]:
model_on_device

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=8640, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=6, bias=True)
)

In [12]:
train_model(80, train_loader, val_loader, criterion, optimizer, device, model_on_device)

Epoch: 1/80 	 Training Loss: 1.2715, Accuracy: 16.87, Testing Loss: 1.7915, Accuracy: 23.97
Epoch: 2/80 	 Training Loss: 1.2706, Accuracy: 18.47, Testing Loss: 1.7916, Accuracy: 29.60
Epoch: 3/80 	 Training Loss: 1.2697, Accuracy: 17.11, Testing Loss: 1.7884, Accuracy: 27.26
Epoch: 4/80 	 Training Loss: 1.2664, Accuracy: 20.07, Testing Loss: 1.7815, Accuracy: 23.08
Epoch: 5/80 	 Training Loss: 1.2466, Accuracy: 23.67, Testing Loss: 1.6756, Accuracy: 37.24
Epoch: 6/80 	 Training Loss: 1.0734, Accuracy: 35.73, Testing Loss: 1.3647, Accuracy: 45.18
Epoch: 7/80 	 Training Loss: 0.9129, Accuracy: 47.24, Testing Loss: 1.2198, Accuracy: 52.73
Epoch: 8/80 	 Training Loss: 0.8159, Accuracy: 51.72, Testing Loss: 1.1230, Accuracy: 52.89
Epoch: 9/80 	 Training Loss: 0.7308, Accuracy: 55.93, Testing Loss: 0.8619, Accuracy: 63.88
Epoch: 10/80 	 Training Loss: 0.6205, Accuracy: 63.86, Testing Loss: 1.1467, Accuracy: 51.07
Epoch: 11/80 	 Training Loss: 0.5583, Accuracy: 66.12, Testing Loss: 0.7569, Ac

In [13]:
torch.save(model_on_device, "model_cnn.pth")

In [ ]:
dummy_input = torch.randn(128, 1, 128, 44, device='cuda')
torch.onnx.export(model_on_device, dummy_input, "output/model.onnx") 

In [52]:
# Evaluate model performance on test data
evaluate_model(test_loader, device, model_on_device, criterion)

(0.23116093925360975, 91.92715411796684)

In [28]:
# Save the model
torch.save(model_on_device, "model.pth")